In [1]:
# coding:utf-8
"""
本篇为单独调试shop()函数的调试记录，后期将加入UserMainData
"""
import pandas as pd
import numpy as np
from impala.dbapi import connect
from impala.util import as_pandas
import cPickle
import re
import datetime as dt
import pymysql.cursors

# Global variables
mysql_host = '172.21.1.151'
mysql_port = 3306
mysql_user = 'xli'
mysql_password = '123456'
mysql_db = 'kdwtemp2'
mysql_charset = 'utf8mb4'

def easy_print(terms,tname, records=10): #逐条描述前五条记录，方便仔细查看表中值的特征
    conn = connect(host='172.21.1.120', port=21050, user='udms', database='kmi_month') 
    cur = conn.cursor()     
    
    sql = 'SELECT ' + terms + ' FROM ' + tname + ' LIMIT 5' 
    cur.execute(sql)
    read_in = as_pandas(cur)    
    cur.close()
    conn.close()
    
    #print in a format
    columns = read_in.columns.values
    values = read_in.values
    records = []
    for v in values:
        records.append(zip(columns, v))
    
    for ix,r in enumerate(records):
        print('----------第%s条记录--------' % (ix+1))
        for k, v in r:
            print('%s: %s' % (k, v))
            
def full_print(terms,tname): #打出全表所选列，以方便进行统计查看规律，决定处理方法
    conn = connect(host='172.21.1.120', port=21050, user='udms', database='kmi_month') 
    cur = conn.cursor()     
    
    sql = 'SELECT ' + terms + ' FROM ' + tname
    cur.execute(sql)
    df = as_pandas(cur)
    print df
    cur.close()
    conn.close()

def is_wine(data):
    pattern = re.compile(r"酒")
    if re.search(pattern,data):
        return 1
    else:
        return 0
    
def is_cigarette(data):
    pattern = re.compile(r"烟")
    if re.search(pattern,data):
        return 1
    else:
        return 0
    
def convert_time(data):
    data = pd.to_datetime(data, format='%Y-%m-%d %H:%M:%S')
    return data

def shop_before_dawn(data):
    if data.hour < 6:
        return 1
    else:
        return 0
    
def shop_morning(data):
    if data.hour >=6 and data.hour < 12:
        return 1
    else:
        return 0
def shop_afternoon(data):
    if data.hour >= 12 and data.hour < 18:
        return 1
    else:
        return 0
    
def shop_night(data):
    if data.hour >= 18:
        return 1
    else:
        return 0

def convert_weekday(data):
    return data.weekday()
    
def shop():
    conn = connect(host='172.21.1.120', port=21050, user='udms', database='kmi_month') 
    cur = conn.cursor()
    
    # 1. 合并商品id和商品类别信息
    sql = 'SELECT materialsortid,materialid,companyid FROM wj_tbl_shopmaterial'
    cur.execute(sql)
    df = as_pandas(cur)
    sql = 'SELECT materialsortid,materialsortname,companyid FROM wj_tbl_shopmaterialsort'
    cur.execute(sql)
    df2 = as_pandas(cur)
    material = pd.merge(df,df2,how='inner',left_on=['materialsortid','companyid'],right_on=['materialsortid','companyid'])
    material.drop('materialsortid',axis=1,inplace=True)
    
    # 2. 订单消费总额和类别判断
    sql = 'SELECT onlineorderid,materialid,companyid,ordernumber,sellprice,originalprice FROM wj_tbl_onlineorderdetail'
    cur.execute(sql)
    df = as_pandas(cur)
    df['sellprice'] = map(int,df['sellprice'])
    df['originalprice'] = map(int,df['originalprice'])
    df['ordernumber'] = map(int,df['ordernumber'])
    df['shop_real_sum'] = df['ordernumber']*df['sellprice']
    df['shop_original_sum'] = df['ordernumber']*df['originalprice']
    # 去除未消费的记录
    df = df[df['shop_original_sum'] > 0]   
    # 判断消费类型
    shop_material = pd.merge(df,material,how='left',left_on=['materialid','companyid'],right_on=['materialid','companyid'])
    shop_material['materialsortname'].fillna('null',inplace=True)
    shop_material['shop_wine'] = [is_wine(i) for i in shop_material['materialsortname']]
    shop_material['shop_cigarette'] = [is_cigarette(i) for i in shop_material['materialsortname']] 
    # 排序和去重，对每个用户取购买烟酒特价最大值
    shop_material.sort_values(by=['onlineorderid','shop_wine','shop_cigarette'],ascending=[1,0,0],inplace=True)
    shop_material.drop_duplicates(subset='onlineorderid',inplace=True)
    
    # 3. 消费静态数据全维度分析统计
    sql = 'SELECT openid,onlineorderid,orderdatetime,status FROM wj_tbl_onlineorder'
    cur.execute(sql)
    df = as_pandas(cur)
    # 只保留已消费订单
    df = df[df['status']=='3']
    # 对单条记录统计用户消费时段
    df['shop_cnt'] = 1
    df['orderdatetime'] = map(convert_time, df['orderdatetime'])
    df['shop_before_dawn'] = map(shop_before_dawn,df['orderdatetime'])
    df['shop_morning'] = map(shop_morning,df['orderdatetime'])
    df['shop_afternoon'] = map(shop_afternoon,df['orderdatetime'])
    df['shop_night'] = map(shop_night,df['orderdatetime'])
    df['weekday'] = map(convert_weekday,df['orderdatetime'])
    df['shop_weekday'] = map(int,df['weekday'].isin({0,1,2,3,4})) # 0是周一，6是周日
    df['shop_weekend'] = map(int,df['weekday'].isin({5,6}))
    df.drop('weekday',axis=1,inplace=True)
    df = pd.merge(shop_material,df,how='outer',left_on='onlineorderid',right_on='onlineorderid')
    df['shop_before_dawn_sum'] = df['shop_before_dawn']*df['shop_original_sum']
    df['shop_morning_sum'] = df['shop_morning']*df['shop_original_sum']
    df['shop_afternoon_sum'] = df['shop_afternoon']*df['shop_original_sum']
    df['shop_night_sum'] = df['shop_night']*df['shop_original_sum']
    df['shop_weekday_sum'] = df['shop_weekday']*df['shop_original_sum']
    df['shop_weekend_sum'] = df['shop_weekend']*df['shop_original_sum']
    
    # 统计用户总消费数据
    df1 = df.groupby('openid',as_index=False).sum()
    drop_columns = ['ordernumber','sellprice','originalprice']
    df1 = df1[df1['shop_real_sum']<=df1['shop_original_sum']]
    df1['shop_real_sum_avg'] = df1['shop_real_sum']/df1['shop_cnt']
    df1['shop_original_sum_avg'] = df1['shop_original_sum']/df1['shop_cnt']
    df1.drop(drop_columns,axis=1,inplace=True)
    df1.columns = ['openid', 'shop_real_sum', 'shop_original_sum','shop_wine_cnt', 'shop_cigarette_cnt','shop_cnt', 'shop_before_dawn_cnt',
       'shop_morning_cnt', 'shop_afternoon_cnt', 'shop_night_cnt', 'shop_weekday_cnt',
       'shop_weekend_cnt', 'shop_before_dawn_sum', 'shop_morning_sum',
       'shop_afternoon_sum', 'shop_night_sum', 'shop_weekday_sum',
       'shop_weekend_sum', 'shop_real_sum_avg', 'shop_original_sum_avg']
    # 统计用户消费最大最小平均值
    # 最大值
    df2 = df.sort_values(['openid','shop_real_sum','shop_original_sum'],ascending=[1,0,0])
    df2.drop_duplicates('openid',inplace=True)
    drop_columns = ['onlineorderid', 'materialid', 'companyid', 'ordernumber',
                   'sellprice', 'originalprice', 
                   'materialsortname', 'shop_wine', 'shop_cigarette', 
                   'orderdatetime', 'status', 'shop_cnt', 'shop_before_dawn', 'shop_morning',
                   'shop_afternoon', 'shop_night', 'shop_weekday', 'shop_weekend',
                   'shop_before_dawn_sum', 'shop_morning_sum', 'shop_afternoon_sum',
                   'shop_night_sum', 'shop_weekday_sum', 'shop_weekend_sum']
    df2.drop(drop_columns,axis=1,inplace=True)
    df2.columns = ['shop_real_sum_max','shop_original_sum_max','openid']
    # 最小值
    df3 = df.sort_values(['openid','shop_real_sum','shop_original_sum'],ascending=[1,1,1])
    df3.drop_duplicates('openid',inplace=True)
    drop_columns = ['onlineorderid', 'materialid', 'companyid', 'ordernumber',
                   'sellprice', 'originalprice', 
                   'materialsortname', 'shop_wine', 'shop_cigarette', 
                   'orderdatetime', 'status', 'shop_cnt', 'shop_before_dawn', 'shop_morning',
                   'shop_afternoon', 'shop_night', 'shop_weekday', 'shop_weekend',
                   'shop_before_dawn_sum', 'shop_morning_sum', 'shop_afternoon_sum',
                   'shop_night_sum', 'shop_weekday_sum', 'shop_weekend_sum']
    df3.drop(drop_columns,axis=1,inplace=True)
    df3.columns = ['shop_real_sum_min','shop_original_sum_min','openid']
    
    df = pd.merge(df2,df3,how='outer',left_on='openid',right_on='openid')
    df = pd.merge(df1,df,how='left',left_on='openid',right_on='openid')
    order_stat = df
    
    # 4. 用户消费最多的一家商家记录
    sql = 'SELECT openid,companyid as shop_companyid FROM wj_tbl_onlineorder'
    cur.execute(sql)
    df = as_pandas(cur)
    df['cnt'] = 1
    df = df.groupby(['openid','shop_companyid'],as_index=False).count()
    df.columns = ['openid','shop_companyid','shop_com_cnt']
    df.sort_values(['openid','shop_com_cnt'],ascending=[1,0],inplace=True)
    df.drop_duplicates(subset='openid',inplace=True)
    shop_com = df
    
    # 5. 对用户在线消费单数计数
    sql = 'SELECT openid,onlineorderid FROM wj_tbl_onlineorder'
    cur.execute(sql)
    df = as_pandas(cur)   
    df = df.groupby('openid',as_index=False).count()
    df.columns = ['openid','shop_order_cnt']
    order_cnt = df
    
    # 6. 合并用户静态消费和其消费最多商户记录
    order_com = pd.merge(order_stat,shop_com,how='inner',left_on='openid',right_on='openid')
    df = pd.merge(order_com,order_cnt,how='inner',left_on='openid',right_on='openid')
    df['shop_cross_com'] = map(int,df['shop_order_cnt'] > df['shop_com_cnt'])
    merged = df
    
    # 7. 和商家数据整合在一起
    sql = """SELECT companyid as shop_companyid,roomnum as shop_com_roomnum,regioncode as shop_com_regioncode,
    managetype as shop_com_managetype,vodsystemtype as shop_com_vodsystemtype FROM kdw_tbl_companybase"""
    cur.execute(sql)
    company = as_pandas(cur)    
    df = pd.merge(merged,company,how='left',left_on='shop_companyid',right_on='shop_companyid')
    
    cur.close()
    conn.close()
    # 将中间结果暂存于Mysqldb中
    conn = pymysql.connect(host=mysql_host,
                                      port=mysql_port,
                                      user=mysql_user,
                                      password=mysql_password,
                                      db=mysql_db,
                                      charset=mysql_charset)
    
    df.to_sql(name="ljl_shop", con=conn, flavor='mysql', if_exists='replace', index=False)
    conn.close()

In [2]:
shop()

D:\Users\Lenovo\Anaconda2\lib\site-packages\pandas\core\generic.py:1003: FutureWarning: The 'mysql' flavor with DBAPI connection is deprecated and will be removed in future versions. MySQL will be further supported with SQLAlchemy connectables.
  dtype=dtype)


## 1. 合并商品id和商品类别信息

In [30]:
df = shop()

In [31]:
df.head()

,materialid,companyid,materialsortname
0,31,1137,食品
1,60,1137,食品
2,296,1137,食品
3,309,1137,食品
4,416,1137,食品


In [28]:
df.shape

(1799, 3)

## 2. 订单消费总额和类别判断

### 原始读出数据

In [37]:
df.head()

,onlineorderid,materialid,companyid,ordernumber,sellprice,originalprice
0,162174,580,4067,1,12.00,12.00
1,162175,774,8005,1,58.00,58.00
2,162175,289,8005,1,0.00,0.00
3,162175,697,8005,1,0.00,0.00
4,162176,308,180,1,16.00,16.00


### 清理未消费数据前

In [85]:
df.head()

,onlineorderid,materialid,companyid,ordernumber,sellprice,originalprice,real_sum,original_sum
0,162174,580,4067,1,12,12,12,12
1,162175,774,8005,1,58,58,58,58
2,162175,289,8005,1,0,0,0,0
3,162175,697,8005,1,0,0,0,0
4,162176,308,180,1,16,16,16,16


### ordernumber类型问题导致刚开始计算总额出错，转为int就好了

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 191543 entries, 0 to 191542
Data columns (total 6 columns):
onlineorderid    191543 non-null object
materialid       191543 non-null object
companyid        191543 non-null object
ordernumber      191543 non-null object
sellprice        191543 non-null int64
originalprice    191543 non-null int64
dtypes: int64(2), object(4)
memory usage: 10.2+ MB


In [86]:
df.shape

(191543, 8)

### 清除未消费记录后

In [91]:
df.head()

,onlineorderid,materialid,companyid,ordernumber,sellprice,originalprice,real_sum,original_sum
0,162174,580,4067,1,12,12,12,12
1,162175,774,8005,1,58,58,58,58
4,162176,308,180,1,16,16,16,16
5,162177,289,4938,1,16,16,16,16
6,162178,1622,1118,1,10,10,10,10


In [92]:
df.shape

(153729, 8)

### shop_sum

In [191]:
df.head()

,onlineorderid,shop_real_sum,shop_original_sum
0,162174,12,12
1,162175,58,58
2,162176,16,16
3,162177,16,16
4,162178,10,10


In [192]:
df.shape

(100216, 3)

### shop_material

In [205]:
df.head()

,onlineorderid,materialid,companyid,ordernumber,sellprice,originalprice,shop_real_sum,shop_original_sum,materialsortname,shop_wine,shop_cigarette
0,162174,580,4067,1,12,12,12,12,null,0,0
1,162175,774,8005,1,58,58,58,58,null,0,0
2,162176,308,180,1,16,16,16,16,null,0,0
3,162177,289,4938,1,16,16,16,16,null,0,0
4,162178,1622,1118,1,10,10,10,10,null,0,0


In [206]:
pd.value_counts(df['onlineorderid'].isnull())

False    100216
Name: onlineorderid, dtype: int64

In [207]:
df.shape

(100216, 11)

## 对每个onlineorderid的消费求总和

### 根据onlineorderid求和后

In [103]:
df.head()

,onlineorderid,ordernumber,sellprice,originalprice,real_sum,original_sum
0,162174,1,12,12,12,12
1,162175,1,58,58,58,58
2,162176,1,16,16,16,16
3,162177,1,16,16,16,16
4,162178,1,10,10,10,10


In [104]:
df.shape

(100216, 6)

### 没变化，故此步无用，弃之

## 判断消费类型

### 判断用户是否购买了烟酒

In [6]:
pd.value_counts(df['shop_wine'])

0    155207
1       258
Name: shop_wine, dtype: int64

In [7]:
pd.value_counts(df['shop_cigarette'])

0    155459
1         6
Name: shop_cigarette, dtype: int64

### 判断用户是否参加抢购活动：此统计不可行

In [ ]:
def is_discount(data):
    pattern1 = re.compile(r"特价")
    pattern2 = re.compile(r"抢购")
    pattern3 = re.compile(r"秒杀")
    pattern4 = re.compile(r"限购")
    if re.search(pattern1,data) or re.search(pattern2,data) or re.search(pattern3,data) or re.search(pattern4,data):
        return 1
    else:
        return 0
shop_material['shop_discount'] = [is_discount(i) for i in shop_material['materialsortname']]

In [20]:
pd.value_counts(df['shop_discount'])

0    155465
Name: shop_discount, dtype: int64

### 排序和去重，对每个用户取购买烟酒行为取最大值

In [30]:
df.head()

,onlineorderid,materialid,companyid,ordernumber,sellprice,originalprice,shop_real_sum,shop_original_sum,materialsortname,shop_wine,shop_cigarette
0,162174,580,4067,1,12,12,12,12,null,0,0
17,162175,774,8005,1,58,58,58,58,null,0,0
123,162176,308,180,1,16,16,16,16,null,0,0
159,162177,289,4938,1,16,16,16,16,null,0,0
277,162178,1622,1118,1,10,10,10,10,null,0,0


In [31]:
df.shape

(100217, 11)

## 3. 消费静态数据全维度分析统计

### df

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100293 entries, 0 to 100292
Data columns (total 27 columns):
onlineorderid           100293 non-null object
materialid              100216 non-null object
companyid               100216 non-null object
ordernumber             100216 non-null float64
sellprice               100216 non-null float64
originalprice           100216 non-null float64
shop_real_sum           100216 non-null float64
shop_original_sum       100216 non-null float64
materialsortname        100216 non-null object
shop_wine               100216 non-null float64
shop_cigarette          100216 non-null float64
openid                  78445 non-null object
orderdatetime           78445 non-null datetime64[ns]
status                  78445 non-null object
shop_cnt                78445 non-null float64
shop_before_dawn        78445 non-null float64
shop_morning            78445 non-null float64
shop_afternoon          78445 non-null float64
shop_night              78445 n

In [4]:
df.shape

(100293, 27)

In [5]:
df.head()

,onlineorderid,materialid,companyid,ordernumber,sellprice,originalprice,shop_real_sum,shop_original_sum,materialsortname,shop_wine,...,shop_afternoon,shop_night,shop_weekday,shop_weekend,shop_before_dawn_sum,shop_morning_sum,shop_afternoon_sum,shop_night_sum,shop_weekday_sum,shop_weekend_sum
0,162174,580,4067,1,12,12,12,12,null,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,162175,774,8005,1,58,58,58,58,null,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,162176,308,180,1,16,16,16,16,null,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,162177,289,4938,1,16,16,16,16,null,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,162178,1622,1118,1,10,10,10,10,null,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### df1

In [8]:
df.head()

,openid,shop_real_sum,shop_original_sum,shop_wine_cnt,shop_cigarette_cnt,shop_cnt,shop_before_dawn_cnt,shop_morning_cnt,shop_afternoon_cnt,shop_night_cnt,shop_weekday_cnt,shop_weekend_cnt,shop_before_dawn_sum,shop_morning_sum,shop_afternoon_sum,shop_night_sum,shop_weekday_sum,shop_weekend_sum,shop_real_sum_avg,shop_original_sum_avg
0,1108,87,88,0,0,1,0,0,1,0,1,0,0,0,88,0,88,0,87.000000,88.00
1,5407216,2,96,0,0,3,0,0,3,0,3,0,0,0,96,0,96,0,0.666667,32.00
2,oHEmguBistfB1lRrOGlpjn9C-6o4,12,17,0,0,4,0,0,3,1,4,0,0,0,16,1,17,0,3.000000,4.25
3,oHEmguPAr3oNWit-3gebzEMkzfNQ,1,1,0,0,1,0,0,0,1,1,0,0,0,0,1,1,0,1.000000,1.00
4,otCz6vg-0gaJ1WXOCR2O0MDu4Q9Q,35,35,0,0,1,1,0,0,0,1,0,35,0,0,0,35,0,35.000000,35.00


In [10]:
df.shape

(49978, 20)

In [11]:
df.columns

Index([u'openid', u'shop_real_sum', u'shop_original_sum', u'shop_wine_cnt',
       u'shop_cigarette_cnt', u'shop_cnt', u'shop_before_dawn_cnt',
       u'shop_morning_cnt', u'shop_afternoon_cnt', u'shop_night_cnt',
       u'shop_weekday_cnt', u'shop_weekend_cnt', u'shop_before_dawn_sum',
       u'shop_morning_sum', u'shop_afternoon_sum', u'shop_night_sum',
       u'shop_weekday_sum', u'shop_weekend_sum', u'shop_real_sum_avg',
       u'shop_original_sum_avg'],
      dtype='object')

### df2 最大值

In [19]:
df.head()

,shop_real_sum_max,shop_original_sum_max,openid
79403,87,88,1108
29906,2,32,5407216
50976,9,10,oHEmguBistfB1lRrOGlpjn9C-6o4
55038,1,1,oHEmguPAr3oNWit-3gebzEMkzfNQ
22212,35,35,otCz6vg-0gaJ1WXOCR2O0MDu4Q9Q


In [20]:
df.shape

(49989, 3)

In [21]:
df.columns

Index([u'shop_real_sum_max', u'shop_original_sum_max', u'openid'], dtype='object')

### df3 最小值

In [25]:
df.head()

,shop_real_sum_min,shop_original_sum_min,openid
79403,87,88,1108
29886,0,32,5407216
55130,1,1,oHEmguBistfB1lRrOGlpjn9C-6o4
55038,1,1,oHEmguPAr3oNWit-3gebzEMkzfNQ
22212,35,35,otCz6vg-0gaJ1WXOCR2O0MDu4Q9Q


In [27]:
df.shape

(49989, 3)

### 合并df2 df3

In [35]:
df.head()

,shop_real_sum_max,shop_original_sum_max,openid,shop_real_sum_min,shop_original_sum_min
0,87,88,1108,87,88
1,2,32,5407216,0,32
2,9,10,oHEmguBistfB1lRrOGlpjn9C-6o4,1,1
3,1,1,oHEmguPAr3oNWit-3gebzEMkzfNQ,1,1
4,35,35,otCz6vg-0gaJ1WXOCR2O0MDu4Q9Q,35,35


In [36]:
df.shape

(49989, 5)

### 和df1合并 得到用户消费数据统计最终表

In [40]:
df.head()

,openid,shop_real_sum,shop_original_sum,shop_wine_cnt,shop_cigarette_cnt,shop_cnt,shop_before_dawn_cnt,shop_morning_cnt,shop_afternoon_cnt,shop_night_cnt,...,shop_afternoon_sum,shop_night_sum,shop_weekday_sum,shop_weekend_sum,shop_real_sum_avg,shop_original_sum_avg,shop_real_sum_max,shop_original_sum_max,shop_real_sum_min,shop_original_sum_min
0,1108,87,88,0,0,1,0,0,1,0,...,88,0,88,0,87.000000,88.00,87,88,87,88
1,5407216,2,96,0,0,3,0,0,3,0,...,96,0,96,0,0.666667,32.00,2,32,0,32
2,oHEmguBistfB1lRrOGlpjn9C-6o4,12,17,0,0,4,0,0,3,1,...,16,1,17,0,3.000000,4.25,9,10,1,1
3,oHEmguPAr3oNWit-3gebzEMkzfNQ,1,1,0,0,1,0,0,0,1,...,0,1,1,0,1.000000,1.00,1,1,1,1
4,otCz6vg-0gaJ1WXOCR2O0MDu4Q9Q,35,35,0,0,1,1,0,0,0,...,0,0,35,0,35.000000,35.00,35,35,35,35


In [41]:
df.shape

(49978, 24)

## 4. 用户消费最多的一家商家记录

In [44]:
df.head()

,openid,shop_companyid,shop_com_cnt
6,-1,4812,6
16,104173,742,1
17,1108,180,4
18,2433209,3753,2
19,276608,742,1


In [45]:
df.shape

(56461, 3)

## 5. 对用户在线消费单数计数

In [48]:
df.head()

,openid,shop_order_cnt
0,-1,26
1,104173,1
2,1108,4
3,2433209,2
4,276608,1


In [49]:
df.shape

(56461, 2)

## 6. 连接三张表并得到在不同商家消费的用户数

In [52]:
df.head()

,openid,shop_real_sum,shop_original_sum,shop_wine_cnt,shop_cigarette_cnt,shop_cnt,shop_before_dawn_cnt,shop_morning_cnt,shop_afternoon_cnt,shop_night_cnt,...,shop_real_sum_avg,shop_original_sum_avg,shop_real_sum_max,shop_original_sum_max,shop_real_sum_min,shop_original_sum_min,shop_companyid,shop_com_cnt,shop_order_cnt,shop_cross_com
0,1108,87,88,0,0,1,0,0,1,0,...,87.000000,88.00,87,88,87,88,180,4,4,0
1,5407216,2,96,0,0,3,0,0,3,0,...,0.666667,32.00,2,32,0,32,3753,40,43,1
2,oHEmguBistfB1lRrOGlpjn9C-6o4,12,17,0,0,4,0,0,3,1,...,3.000000,4.25,9,10,1,1,10233,7,7,0
3,oHEmguPAr3oNWit-3gebzEMkzfNQ,1,1,0,0,1,0,0,0,1,...,1.000000,1.00,1,1,1,1,10233,1,1,0
4,otCz6vg-0gaJ1WXOCR2O0MDu4Q9Q,35,35,0,0,1,1,0,0,0,...,35.000000,35.00,35,35,35,35,6195,1,1,0


In [53]:
df.shape

(49978, 28)

In [54]:
pd.value_counts(df['shop_cross_com'])

0    49791
1      187
Name: shop_cross_com, dtype: int64

## 7. 和商家数据整合在一起

In [59]:
df.head()

,openid,shop_real_sum,shop_original_sum,shop_wine_cnt,shop_cigarette_cnt,shop_cnt,shop_before_dawn_cnt,shop_morning_cnt,shop_afternoon_cnt,shop_night_cnt,...,shop_real_sum_min,shop_original_sum_min,shop_companyid,shop_com_cnt,shop_order_cnt,shop_cross_com,shop_com_roomnum,shop_com_regioncode,shop_com_managetype,shop_com_vodsystemtype
0,1108,87,88,0,0,1,0,0,1,0,...,87,88,180,4,4,0,55,350000,1,0
1,5407216,2,96,0,0,3,0,0,3,0,...,0,32,3753,40,43,1,NaN,NaN,NaN,NaN
2,oHEmguBistfB1lRrOGlpjn9C-6o4,12,17,0,0,4,0,0,3,1,...,1,1,10233,7,7,0,NaN,NaN,NaN,NaN
3,oHEmguPAr3oNWit-3gebzEMkzfNQ,1,1,0,0,1,0,0,0,1,...,1,1,10233,1,1,0,NaN,NaN,NaN,NaN
4,otCz6vg-0gaJ1WXOCR2O0MDu4Q9Q,35,35,0,0,1,1,0,0,0,...,35,35,6195,1,1,0,46,440000,1,0


In [60]:
df.shape

(49978, 32)

In [3]:
df.columns

Index([u'openid', u'shop_real_sum', u'shop_original_sum', u'shop_wine_cnt',
       u'shop_cigarette_cnt', u'shop_cnt', u'shop_before_dawn_cnt',
       u'shop_morning_cnt', u'shop_afternoon_cnt', u'shop_night_cnt',
       u'shop_weekday_cnt', u'shop_weekend_cnt', u'shop_before_dawn_sum',
       u'shop_morning_sum', u'shop_afternoon_sum', u'shop_night_sum',
       u'shop_weekday_sum', u'shop_weekend_sum', u'shop_real_sum_avg',
       u'shop_original_sum_avg', u'shop_real_sum_max',
       u'shop_original_sum_max', u'shop_real_sum_min',
       u'shop_original_sum_min', u'shop_companyid', u'shop_com_cnt',
       u'shop_order_cnt', u'shop_cross_com', u'shop_com_roomnum',
       u'shop_com_regioncode', u'shop_com_managetype',
       u'shop_com_vodsystemtype'],
      dtype='object')

In [6]:
for i in df.columns:
    print i+"  "

openid  
shop_real_sum  
shop_original_sum  
shop_wine_cnt  
shop_cigarette_cnt  
shop_cnt  
shop_before_dawn_cnt  
shop_morning_cnt  
shop_afternoon_cnt  
shop_night_cnt  
shop_weekday_cnt  
shop_weekend_cnt  
shop_before_dawn_sum  
shop_morning_sum  
shop_afternoon_sum  
shop_night_sum  
shop_weekday_sum  
shop_weekend_sum  
shop_real_sum_avg  
shop_original_sum_avg  
shop_real_sum_max  
shop_original_sum_max  
shop_real_sum_min  
shop_original_sum_min  
shop_companyid  
shop_com_cnt  
shop_order_cnt  
shop_cross_com  
shop_com_roomnum  
shop_com_regioncode  
shop_com_managetype  
shop_com_vodsystemtype  
